#### Setup + Prepare Data

In [1]:
# Setup 
# !pip3 install pandas
import pandas as pd
import numpy as np
edges = pd.read_csv('edges.tsv', sep='\t')
nodes = pd.read_csv('nodes.tsv', sep='\t')

/var/folders/v2/09vly0y934v7rq6zkxt1fwvr0000gn/T/ipykernel_39782/1778872367.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/var/folders/v2/09vly0y934v7rq6zkxt1fwvr0000gn/T/ipykernel_39782/1778872367.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  edges = pd.read_csv('edges.tsv', sep='\t')


Get a map of relations we want and possible objects (get a edges file we specifically want?)

In [84]:
columns = [
    'Contact', 'Dominant_end_handshape', 'Dominant_start_handshape',
    'Flexion', 'Flexion_Change', 'Handshape', 'Location', 'Major_Location',
    'Minor_Location', 'Nondominant_Handshape', 'Path_Movement',
    'Repeated_Movement', 'Second_Minor_Location', 'Selected_Fingers',
    'Sign_Duration', 'Sign_Type', 'Spread', 'Spread_Change',
    'Thumb_Contact', 'Thumb_Position', 'Wrist_Twist', 'sign_type'
]

In [85]:
cols_keywords = {}

for column in columns: 
    set_objs = set(edges[edges['relation'] == column]['object'])
    cols_keywords[column] = set_objs
   
# print(cols_keywords)

Update edges file

In [86]:
filteredEdges = edges[edges["relation"].isin(columns)]

change 1s and 0s to true and false

In [87]:
filteredEdges.loc[(filteredEdges['object'] == '1.0') & 
                  (~filteredEdges['relation'].isin(['Handshape', 'Dominant_end_handshape', 'Dominant_start_handshape', 'Nondominant_Handshape'])), 
                  ['object']] = True
filteredEdges.loc[(filteredEdges['object'] == '1') & 
                  (~filteredEdges['relation'].isin(['Handshape', 'Dominant_end_handshape', 'Dominant_start_handshape', 'Nondominant_Handshape'])), 
                  ['object']] = True
filteredEdges.loc[(filteredEdges['object'] == '0.0') & 
                  (~filteredEdges['relation'].isin(['Handshape', 'Dominant_end_handshape', 'Dominant_start_handshape', 'Nondominant_Handshape'])), 
                  ['object']] = False
filteredEdges.loc[(filteredEdges['object'] == '0') & 
                  (~filteredEdges['relation'].isin(['Handshape', 'Dominant_end_handshape', 'Dominant_start_handshape', 'Nondominant_Handshape'])), 
                  ['object']] = False

change the numerical values of into ranges (Sign_Duration)

In [88]:
#for all sign duration values, round down to nearest 100 to get a 'range'
sign_duration_rows = filteredEdges[filteredEdges['relation'] == 'Sign_Duration']
sign_duration_rows['object'] = pd.to_numeric(sign_duration_rows['object'], errors='coerce')
sign_duration_rows.dropna(subset=['object'], inplace=True)
sign_duration_rows['object'] = (sign_duration_rows['object'] // 100) * 100
sign_duration_rows['object'] = sign_duration_rows['object'].astype(int).astype(str)
filteredEdges.loc[sign_duration_rows.index, 'object'] = sign_duration_rows['object']

/var/folders/v2/09vly0y934v7rq6zkxt1fwvr0000gn/T/ipykernel_26099/1471796543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sign_duration_rows['object'] = pd.to_numeric(sign_duration_rows['object'], errors='coerce')
/var/folders/v2/09vly0y934v7rq6zkxt1fwvr0000gn/T/ipykernel_26099/1471796543.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sign_duration_rows.dropna(subset=['object'], inplace=True)
/var/folders/v2/09vly0y934v7rq6zkxt1fwvr0000gn/T/ipykernel_26099/1471796543.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

Keep only the columns I want

In [89]:
filteredEdges = filteredEdges.loc[:, ['subject', 'relation', 'object']]

In [90]:
filteredEdges['subject_count'] = filteredEdges.groupby('subject')['subject'].transform('count')

In [91]:
filteredEdges
filteredEdges.to_csv('filteredEdges.csv', index=False)

In [92]:
filtered_cols_keywords = {}

for column in columns: 
    set_objs = set(filteredEdges[filteredEdges['relation'] == column]['object'])
    filtered_cols_keywords[column] = set_objs

In [93]:
filtered_cols_keywords
np.save('filtered_edges_dict.npy', filtered_cols_keywords) 

In [94]:
keywords_df = pd.DataFrame.from_dict(filtered_cols_keywords, orient='index')
keywords_df.to_csv('keywords_df.csv', index=True, index_label='relation')

#### Model

##### Recieve a list of phonological features

In [4]:
import pandas as pd

/var/folders/v2/09vly0y934v7rq6zkxt1fwvr0000gn/T/ipykernel_12378/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
edges = pd.read_csv('filteredEdges.csv')

In [14]:
test_sample = edges[edges['subject'] == 'TV']

In [20]:
selected_keywords = []
for index, row in test_sample.iterrows():
    selected_keywords.append({'selected_relation': row['relation'], 'selected_object': row['object']})


In [21]:
selected_keywords

[{'selected_relation': 'Sign_Duration', 'selected_object': '500'},
 {'selected_relation': 'Handshape', 'selected_object': 'baby_o'},
 {'selected_relation': 'Selected_Fingers', 'selected_object': 'i'},
 {'selected_relation': 'Flexion', 'selected_object': 'bent'},
 {'selected_relation': 'Thumb_Position', 'selected_object': 'open'},
 {'selected_relation': 'Thumb_Contact', 'selected_object': 'True'},
 {'selected_relation': 'Sign_Type', 'selected_object': 'one handed'},
 {'selected_relation': 'Path_Movement', 'selected_object': 'none'},
 {'selected_relation': 'Repeated_Movement', 'selected_object': 'False'},
 {'selected_relation': 'Major_Location', 'selected_object': 'neutral'},
 {'selected_relation': 'Minor_Location', 'selected_object': 'neutral'},
 {'selected_relation': 'Contact', 'selected_object': 'False'},
 {'selected_relation': 'Wrist_Twist', 'selected_object': 'False'}]

##### For each sign, get a set of signs at which that relation = true

In [33]:
setOfSigns = pd.DataFrame()
toConcat = []

for i, row in enumerate(selected_keywords):
    thisRelation = row['selected_relation']
    thisObject = row['selected_object']
    signsWithThose = edges[edges['relation'] == thisRelation][edges['object'] == thisObject]
    toConcat.append(signsWithThose)

setOfSigns = pd.concat(toConcat, ignore_index=True)

/var/folders/v2/09vly0y934v7rq6zkxt1fwvr0000gn/T/ipykernel_12378/708101150.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  signsWithThose = edges[edges['relation'] == thisRelation][edges['object'] == thisObject]
/var/folders/v2/09vly0y934v7rq6zkxt1fwvr0000gn/T/ipykernel_12378/708101150.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  signsWithThose = edges[edges['relation'] == thisRelation][edges['object'] == thisObject]
/var/folders/v2/09vly0y934v7rq6zkxt1fwvr0000gn/T/ipykernel_12378/708101150.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  signsWithThose = edges[edges['relation'] == thisRelation][edges['object'] == thisObject]
/var/folders/v2/09vly0y934v7rq6zkxt1fwvr0000gn/T/ipykernel_12378/708101150.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  signsWithThose = edges[edges['relation'] == thisRelation][edges['object'] == thisObject]
/var/fol

In [35]:
counts = setOfSigns['subject'].value_counts()
setOfSigns['repeat_count'] = setOfSigns['subject'].map(counts)

In [37]:
setOfSigns['match_proportion'] = setOfSigns['repeat_count'] / setOfSigns['subject_count']

In [38]:
setOfSigns

,subject,relation,object,subject_count,repeat_count,match_proportion
0,these,Sign_Duration,500,27,12,0.444444
1,book,Sign_Duration,500,30,5,0.166667
2,apple,Sign_Duration,500,26,6,0.230769
3,must,Sign_Duration,500,25,13,0.520000
4,milk,Sign_Duration,500,27,13,0.481481
...,...,...,...,...,...,...
18053,zero,Wrist_Twist,False,29,12,0.413793
18054,zipper,Wrist_Twist,False,26,10,0.384615
18055,zoo,Wrist_Twist,False,20,10,0.500000
18056,zoom_in,Wrist_Twist,False,22,12,0.545455


In [39]:
max_index = setOfSigns['match_proportion'].idxmax()
highest_ratio_row = setOfSigns.loc[max_index]

In [42]:
predicted_subject = highest_ratio_row['subject']

##### Get an ordered list of signs with descending # of keyword matches

##### Get & display gloss and #matches for top n signs